In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GroupKFold
## Custom libraries
import index_helpers as ih
from data_transformations import transformation_call
from sklearn.metrics import roc_auc_score

In [3]:
#choice of segmentation
segmentation = True
fine_segmentation=True

#Try Expert, Metadata, Normalization, Power
exclude_expert=False
exclude_meta_data=True
normalization=False
power=False
## Import, index, and split
df = ih.read_and_merge_data(segmentation, fine_segmentation, exclude_expert, exclude_meta_data)
df = transformation_call(df, normalization, power)
df = ih.index_df_by_person(df)
df.head()

Label  EEPD50_100  \
File_Name_split                      File_n_recording                      
008ba489-31ad-44d8-856b-fcf72369dc46 0                     1         6.0   
                                     1                     1         3.0   
                                     2                     1         4.0   
                                     3                     1         4.0   
                                     4                     1         4.0   

                                                       EEPD100_150  \
File_Name_split                      File_n_recording                
008ba489-31ad-44d8-856b-fcf72369dc46 0                         6.0   
                                     1                         3.0   
                                     2                         4.0   
                                     3                         4.0   
                                     4                         4.0   

                                                       EEPD150_200  \
File_Name_split                      File_n_recording                
008ba489-31ad-44d8-856b-fcf72369dc46 0                         5.0   
                                     1                         3.0   
                                     2                         3.0   
                                     3                         4.0   
                                     4                         5.0   

                                                       EEPD200_250  \
File_Name_split                      File_n_recording                
008ba489-31ad-44d8-856b-fcf72369dc46 0                         4.0   
                                     1                         4.0   
                                     2                         4.0   
                                     3                         4.0   
                                     4                         4.0   

                                                       EEPD250_300  \
File_Name_split                      File_n_recording                
008ba489-31ad-44d8-856b-fcf72369dc46 0                         5.0   
                                     1                         4.0   
                                     2                         3.0   
                                     3                         4.0   
                                     4                         4.0   

                                                       EEPD300_350  \
File_Name_split                      File_n_recording                
008ba489-31ad-44d8-856b-fcf72369dc46 0                         5.0   
                                     1                         5.0   
                                     2                         3.0   
                                     3                         5.0   
                                     4                         5.0   

                                                       EEPD350_400  \
File_Name_split                      File_n_recording                
008ba489-31ad-44d8-856b-fcf72369dc46 0                         6.0   
                                     1                         4.0   
                                     2                         4.0   
                                     3                         3.0   
                                     4                         3.0   

                                                       EEPD400_450  \
File_Name_split                      File_n_recording                
008ba489-31ad-44d8-856b-fcf72369dc46 0                         6.0   
                                     1                         4.0   
                                     2                         4.0   
                                     3                         4.0   
                                     4                         4.0   

                                                       EEPD450_500  ...  \
File_Name_

In [4]:
df = ih.categorical_float_to_int(df)
df = ih.categorical_to_dummy(df)
X_train, X_val, y_train, y_val = ih.train_test_split_on_index(features = df.drop('Label', axis=1),
                                                             label = df["Label"])

## Modify data for GroupKFold
groups = y_train.reset_index()['File_Name_split']
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

In [7]:
X_train.head()

,Zero_Crossing_Rate,RMS_Power,Dominant_Freq,Spectral_Centroid,Spectral_Rolloff,Spectral_Spread,Spectral_Skewness,Spectral_Kurtosis,Spectral_Bandwidth,Spectral_Flatness,...,EEPD950_1000_6,EEPD950_1000_7,EEPD950_1000_8,EEPD950_1000_9,EEPD950_1000_10,EEPD950_1000_11,EEPD950_1000_12,EEPD950_1000_13,EEPD950_1000_15,EEPD950_1000_17
0,0.160222,0.128937,0.031250,1773.765792,3415.0,1422.780346,0.758778,2.332231,262068.236944,0.081322,...,0,0,0,0,0,0,0,0,0,0
1,0.157047,0.092217,0.062500,1801.097694,2501.0,1486.201986,0.657349,2.000908,183434.937753,0.083739,...,0,0,0,0,0,0,0,0,0,0
2,0.154893,0.075413,0.046875,1589.223922,2522.0,1294.383203,1.037579,3.070197,137156.195637,0.043211,...,0,0,0,0,0,0,0,0,0,0
3,0.150417,0.049969,0.046875,1476.036555,2273.0,1301.461039,1.146937,3.091304,101036.321780,0.031093,...,0,0,0,0,0,0,0,0,0,0
4,0.151335,0.049786,0.050781,1478.032136,2298.0,1299.889388,1.138023,3.063961,101436.817158,0.032721,...,0,0,0,0,0,0,0,0,0,0


In [8]:
### Logistic regression
from sklearn.linear_model import LogisticRegression

## Make pipeline - name classifier "clf"
clf_pipeline = Pipeline([("st_scaler", StandardScaler()),
                        ("clf", LogisticRegression())])

## Use "clf__" in order to correctly assign parameters to the clf object
clf_param_grid = {'clf__penalty':['l1', 'l2'],
                  'clf__tol':[ 0.0001], 
                  'clf__C':[2.0, 1.5, 1.0], 
                  'clf__intercept_scaling':[1], 
                  'clf__max_iter':[200],
                 'clf__solver':[ 'lbfgs', 'liblinear', 'sag', 'saga']}

## Instantiate GroupKFold to avoid data leakage - to be passed to cv
gkf=GroupKFold(n_splits=10)

## Set up Randomized search CV
clf_rand_auc = RandomizedSearchCV(estimator=clf_pipeline,
                                  param_distributions=clf_param_grid,
                                  cv=gkf, scoring='roc_auc', verbose=1, n_jobs=2, n_iter=25)

## Perform Group K-Cross-validation
clf_rand_auc.fit(X_train, y_train, groups=groups)
pred=clf_rand_auc. predict(X_val)
auc=roc_auc_score(pred, y_val)

print('Test AUC :', auc)
## Print results
print("Best score: ",  clf_rand_auc.best_score_)
print("Best estimator: ", clf_rand_auc.best_estimator_)

C:\Users\chris\anaconda\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 24 is smaller than n_iter=25. Running 24 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 10 folds for each of 24 candidates, totalling 240 fits


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  1.0min
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:  5.8min
[Parallel(n_jobs=2)]: Done 240 out of 240 | elapsed:  7.4min finished


Test AUC : 0.6180570462595256
Best score:  0.5714223961179993
Best estimator:  Pipeline(steps=[('st_scaler', StandardScaler()),
                ('clf',
                 LogisticRegression(max_iter=200, penalty='l1',
                                    solver='saga'))])


C:\Users\chris\anaconda\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
